In [1]:
%pip install pythautomata

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 1.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Creating a moore machine

#### Dependencies

In [2]:
from pythautomata.base_types.alphabet import Alphabet
from pythautomata.base_types.moore_state import MooreState
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.automata.moore_machine_automaton import MooreMachineAutomaton
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy as ComparisonStrategy
from pythautomata.model_exporters.dot_exporting_mm_strategy import DotExportingMMStrategy

#### Implementation

In [3]:
# Input alphabet with symbols a, b and c
input_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'), SymbolStr('c'))))
# Output alphabet with symbols 0 and 1
output_alphabet = Alphabet(frozenset((SymbolStr('0'), SymbolStr('1'))))

a = input_alphabet['a']
b = input_alphabet['b']
c = input_alphabet['c']

# Define states
stateA = MooreState("State A", output_alphabet['0'])
stateB = MooreState("State B", output_alphabet['1'])
stateC = MooreState("State C", output_alphabet['1'])

# Add transitions
stateA.add_transition(a, stateA)
stateA.add_transition(b, stateB)
stateB.add_transition(a, stateC)
stateB.add_transition(c, stateB)

# Hole state
hole_state = MooreState(name="hole", value=SymbolStr('0'))

moore_machine = MooreMachineAutomaton(input_alphabet, output_alphabet, stateA, set([stateA, stateB, stateC]), ComparisonStrategy, "moore machine with 3 states", [DotExportingMMStrategy])

# Moore machine is generated in the directory ./output_models/moore_machines/
moore_machine.export('./output_models/moore_machines/')

Unexpected exception when exporting moore machine with 3 states: <class 'AttributeError'>


### Moore machine generator

#### Dependencies

In [4]:
from pythautomata.utilities.nicaud_mm_generator import generate_moore_machine

#### Implementation

In [5]:
# Amount of states
size = 20

generated_moore_machine = generate_moore_machine(input_alphabet, output_alphabet, size)

# Moore machine is generated in the directory ./output_models/moore_machines/
generated_moore_machine.export('./output_models/moore_machines/')

### DFA - moore machine comparator and converter

#### Dependencies

In [6]:
from pythautomata.utilities.automata_converter import AutomataConverter
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines

#### Implementation

In [7]:
# Tomitas DFA example
tomitas = TomitasGrammars.get_automaton_1()

# Convert DFA to moore machine
converted_moore_machine = AutomataConverter.convert_dfa_to_moore_machine(tomitas)

# Tomitas moore machine
tomitas_moore_machine = SampleMooreMachines.get_tomitas_automaton_1()

# Comparate automatons
converted_moore_machine.__eq__(tomitas_moore_machine)

True

In [8]:
%pip install pymodelextractor

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 1.7 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: pythautomata
    Found existing installation: pythautomata 0.21.2
    Uninstalling pythautomata-0.21.2:
      Successfully uninstalled pythautomata-0.21.2

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Moore machine L*

#### Dependencies

In [9]:
from pythautomata.automata.moore_machine_automaton import MooreMachineAutomaton
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy
from pymodelextractor.learners.observation_table_learners.mm_lstar_learner import MMLStarLearner as MooreMachineLearner
from pymodelextractor.teachers.moore_machines_teacher import MooreMachineTeacher as MMTeacher
from pythautomata.utilities.nicaud_mm_generator import generate_moore_machine
from pythautomata.base_types.alphabet import Alphabet


#### Implementation

In [11]:

# Obtain the Moore Machine Learner that implements the L* algorithm
def get_mm_learner():
    return MooreMachineLearner()

# Obtain the teacher for the Moore Machine Learner
def get_mm_teacher(automaton: MooreMachineAutomaton) -> MMTeacher:
    return MMTeacher(automaton, MooreMachineComparisonStrategy())

def test_500_states_moore_machine():
    # Generate a random Moore Machine with 500 states
    input_alphabet = Alphabet.from_strings(["a", "b", "c"])
    output_alphabet = Alphabet.from_strings(['0', '1'])
    seed = 100
    automata = generate_moore_machine(input_alphabet, output_alphabet, 500, seed)

    # Learn the generated Moore Machine
    teacher = get_mm_teacher(automata)
    result = get_mm_learner().learn(teacher, verbose=True) # verbose=True prints the learning process
    
    # Show that the learned Moore Machine is equivalent to the generated Moore Machine
    print("Are equal: ", MooreMachineComparisonStrategy().are_equivalent(
        result.model, automata))

    # Export the learned Moore Machine
    result.model.export('./output_models/mm_lstar/')


test_500_states_moore_machine()

**** Started moore machines lstar ****
 # Starting iteration 1
    . Closed table in 0.00022029876708984375s
    + Made table consistent in 1.5735626220703125e-05s
    - Found counterexample in 0.0003833770751953125s -> ab
  # Iteration 1 ended, duration: 0.001489877700805664s
 # Starting iteration 2
    . Closed table in 3.5762786865234375e-05s
    . Closed table in 0.0004119873046875s
    . Closed table in 0.0013873577117919922s
    . Closed table in 0.0048885345458984375s
    . Closed table in 0.011749029159545898s
    . Closed table in 0.12101387977600098s
    . Closed table in 0.2936675548553467s
    . Closed table in 0.8626790046691895s
    . Closed table in 1.6988496780395508s
    . Closed table in 1.2561454772949219s
    . Closed table in 1.8004918098449707s
    . Closed table in 3.203772783279419s
    . Closed table in 4.451939344406128s
    . Closed table in 4.867054462432861s
    . Closed table in 6.982788801193237s
    . Closed table in 11.599337100982666s
    . Closed tabl